In [2]:
import pandahouse as ph

In [3]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }
connection_test = dict(database='test',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student-rw',
                  password='656e2b0c9c')

In [4]:
Query1 = '''
WITH aboba AS (
    SELECT
        st_id AS hard_workers,
        COUNT(correct) AS answers
    FROM default.peas
    WHERE correct = 1 AND timest BETWEEN '2021-10-01' AND '2021-10-31'
    GROUP BY st_id
    HAVING answers >= 20)
SELECT
    COUNT(hard_workers) AS hard_workers_amount
FROM aboba'''

### Пояснение первого запроса

Сперва создаю временную таблицу, в которой сохраняю сгруппированную по пользователям информацию о количестве решенных задач. В WHERE исключаю все неверное решенные задачи (ибо для нас они не имеют значения) и ограничиваем временные рамки (на самом деле, конкретно тут это не то, чтобы имело большой смысл, ибо данные присутствуют только за 30 и 31 октября 2021 года, но исхожу из предположения масштабирования задания в будущем при разрастании таблицы). Затем в HAVING исключаю всех пользователей, которые решили меньше 20 "горошин" за месяц, поскольку это была нижняя граница "очень усердного ученика"

В конце просто подсчитываю количество id, которые остались после всех ограничений

In [5]:
q1 = ph.read_clickhouse(query = Query1, connection = connection_default)

In [6]:
q1

,hard_workers_amount
0,30


In [19]:
Query2 = '''
WITH ABOBA AS (
    SELECT 
        DISTINCT L.st_id AS st_id, 
        R.test_grp AS TestGroup
    FROM default.peas L
    LEFT JOIN default.studs R ON L.st_id = R.st_id
),
ABOBIK AS (
    SELECT 
        st_id,
        SUM(money) AS total_money
    FROM default.final_project_check
    WHERE sale_time BETWEEN '2021-10-01' AND '2021-10-31'
    GROUP BY st_id
),
ACTIVE AS (
    SELECT 
        st_id,
        COUNT(*) AS correct_tasks
    FROM default.peas
    WHERE correct = 1
    GROUP BY st_id
    HAVING SUM(correct) >= 10),
MATH AS (
    SELECT
        st_id,
        COUNT(*) AS correct_tasks
    FROM default.peas
    WHERE correct = 1 AND subject == 'Math'
    GROUP BY st_id
    HAVING COUNT(correct) >= 2),
MATH2 AS (
    SELECT
        st_id,
        SUM(money) AS total_money
    FROM default.final_project_check
    WHERE subject = 'Math'
    GROUP BY st_id)

SELECT 
    L.TestGroup AS TestGroup,
    SUM(R.total_money) / COUNT(DISTINCT L.st_id) AS ARPU,
    SUM(R.total_money) / COUNT(DISTINCT N.st_id) AS ARPAU,
    COUNT(DISTINCT R.st_id)/COUNT(DISTINCT L.st_id) AS CR,
    COUNT(DISTINCT R.st_id)/COUNT(DISTINCT N.st_id) AS CR_active,
    COUNT(DISTINCT Y.st_id)/COUNT(DISTINCT M.st_id) AS CR_math
FROM ABOBA L
LEFT JOIN ABOBIK R ON L.st_id = R.st_id
LEFT JOIN ACTIVE N ON L.st_id = N.st_id
LEFT JOIN MATH M ON L.st_id = M.st_id
LEFT JOIN MATH2 Y ON L.st_id = Y.st_id
GROUP BY L.TestGroup'''

### Пояснение второго запроса

**Временная таблица ABOBA**: Я сохраняю в ней только уникальные id пользователей, которые есть в таблице default.peas, дабы не разбирать пользователей, которые за указанный период не решали "горошин", а также присвоил каждому пользователю соответствующую группу

**Временная таблица ABOBIK**: Я сгруппировал по id всех пользовтелей и посчитал суммарно потраченную сумму на каждого из них. Таблица будет мне необходима для подсчета ARPU, так как благодаря ней я буду учитывать всю полученную сумму от всех пользователей

**Временная таблица ACTIVE**: Я группирую данные по id пользователей и считаю для каждого число решенных "горошин", отфильтровывая неправильно решенные "горошины" и пользователей, решивших меньше 10 задач (условия активного пользователя). Использую эту таблицу для определения общего числа активных пользователей, что использую для подсчета ARPAU

**Временная таблица MATH**: Действия аналогичны таблице ARPAU, но в этот раз дополнительно прописываю необходимый предмет (Math) и изменяю число решенных "горошин" (2)

**Временная таблица MATH2**: Таблица аналогична ABOBIK, кроме дополнительного условия - предмет Math. Однако он используется не для того, чтобы посчитать сумму, а чтобы найти число пользователей, которые купили курс именно по математике. Обе последние таблицы используются для подсчета CR в математику

#### Рассчет метрик

**ARPU** - суммарное число полученных денег (из таблицы ABOBIK) деленное на количество всех пользователей (из таблицы ABOBA)

**ARPAU** - суммарное число полученных денег (из таблицы ABOBIK) деленное на количество всех активных пользователей (из таблицы ACTIVE)

**CR** - число купивших курс (из таблицы ABOBIK) деленное на число всех пользователей, решавших горошины (из таблицы ABOBA)

**CR_active** - число купивших курс (из таблицы ABOBIK) деленное на число всех активных пользователей (из таблицы ACTIVE)

**CR_math** - число купивших курс по математике (из таблицы MATH2) деленное на число всех активных пользователей по математике (из таблицы MATH)

In [20]:
q2 = ph.read_clickhouse(query = Query2, connection = connection_default)

In [21]:
q2

,TestGroup,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4242.424242,5263.157895,0.054545,0.067669,0.180000
1,pilot,6223.776224,8725.490196,0.069930,0.098039,0.395349
